In [ ]:
import numpy as np
import sympy
import qsymm

sympy.init_printing(print_builtin=True)
np.set_printoptions(precision=2, suppress=True, linewidth=150)

# Find symmetries

Define Rashba-Hamiltonian

In [ ]:
ham1 = ("hbar^2 / (2 * m) * (k_x**2 + k_y**2 + k_z**2) * eye(2) +" +
        "alpha * sigma_x * k_x + alpha * sigma_y * k_y + alpha * sigma_z * k_z")
# Convert to standard monomials form
H1 = qsymm.Model(ham1)
H1.tosympy()

Define `cubic_group` as the set of candidates for point group symmetries.

In [ ]:
cubic_group = qsymm.groups.cubic()

Find point group symmetries, and discrete onsite and continuous symmetries.

In [ ]:
sg, cg = qsymm.symmetries(H1, cubic_group)
print(len(sg))
print(cg)

Print the names of the group elements. It includes time reversal, but not inversion.

In [ ]:
print([qsymm.groups.name_PG_elements(g) for g in sg])

Check taht this is the same as the full cubic group without inversion plus TR.

In [ ]:
C4 = qsymm.PointGroupElement(np.array([[1, 0, 0],
                                       [0, 0, 1],
                                       [0, -1, 0]]), False, False, None)
C3 = qsymm.PointGroupElement(np.array([[0, 0, 1],
                                       [1, 0, 0],
                                       [0, 1, 0]]), False, False, None)
TR = qsymm.PointGroupElement(np.eye(3), True, False, None)

set(sg) == qsymm.groups.generate_group({C4, C3, TR})

Add degeneracy, adds an `SU(2)` continuous symmetry group.

In [ ]:
ham2 = "kron(eye(2), " + ham1 + ")"
print(ham2)
# Convert to standard monomials form
H2 = qsymm.Model(ham2)
display(H2.tosympy())
sg, cg = qsymm.symmetries(H2, cubic_group)
print(len(sg))
print(cg)

Add hole degrees of freedom instead, only adds `U(1)` charge conservation, as there is no pairing.

In [ ]:
ham3 = "kron(sigma_z, " + ham1 + ")"
print(ham3)
# Convert to standard monomials form
H3 = qsymm.Model(ham3)
display(H3.tosympy())
sg, cg = qsymm.symmetries(H3, cubic_group)
print(len(sg))
print(cg)

# Bloch Hamiltonian

Define hexagonal point group in 2D

In [ ]:
hex_group_2D = qsymm.groups.hexagonal()
print(len(hex_group_2D))

Single band nearest neighbor hopping in triangular lattice

In [ ]:
ham6 = 'm * (cos(k_x) + cos(1/2*k_x + sqrt(3)/2*k_y) + cos(-1/2*k_x + sqrt(3)/2*k_y))'
display(qsymm.sympify(ham6))
H6 = qsymm.Model(ham6, momenta=[0, 1])
sg, cg = qsymm.symmetries(H6, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
print(cg)

Add spin and Rashba term

In [ ]:
ham62 = 'eye(2) * (' + ham6 + ') +'
ham62 += 'alpha * (sin(k_x) * sigma_x + sin(1/2*k_x + sqrt(3)/2*k_y) * (1/2*sigma_x + sqrt(3)/2*sigma_y) +'
ham62 += 'sin(-1/2*k_x + sqrt(3)/2*k_y) * (-1/2*sigma_x + sqrt(3)/2*sigma_y))'
print(ham62)
display(qsymm.sympify(ham62))
H62 = qsymm.Model(ham62, momenta=[0, 1])
sg, cg = qsymm.symmetries(H62, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
print(cg)

Add degeneracy

In [ ]:
ham63 = 'kron(eye(2), ' + ham62 + ')'
display(qsymm.sympify(ham63))
H63 = qsymm.Model(ham63, momenta=[0, 1])
sg, cg = qsymm.symmetries(H63, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
print(cg)

Add hole degrees of freedom

In [ ]:
ham64 = 'kron(sigma_z, ' + ham62 + ')'
print(ham64)
display(qsymm.sympify(ham64))
H64 = qsymm.Model(ham64, momenta=[0, 1])
sg, cg = qsymm.symmetries(H64, hex_group_2D)
print(len(sg))
print(set(sg) == hex_group_2D)
print(cg)

# Continuous Rotation Symmetry

Rashba Hamiltonian actually has full rotation invariance.

In [ ]:
display(qsymm.sympify(ham1))
qsymm.symmetries(H1, continuous_rotations=True, prettify=True)

# More complicated examples
## k.p Hamiltonian

Use an 8x8 k.p Hamiltonian of zinc blende semiconductors.

In [ ]:
serialized_kp = 'Matrix([[hbar**2*k_x*gamma_0*k_x/(2*m_0)+hbar**2*k_y*gamma_0*k_y/(2*m_0)+hbar**2*k_z*gamma_0*k_z/(2*m_0)+E_0+E_v,0,-sqrt(2)*P*k_x/2-sqrt(2)*I*P*k_y/2,sqrt(6)*P*k_z/3,sqrt(6)*P*k_x/6-sqrt(6)*I*P*k_y/6,0,-sqrt(3)*P*k_z/3,-sqrt(3)*P*k_x/3+sqrt(3)*I*P*k_y/3],[0,hbar**2*k_x*gamma_0*k_x/(2*m_0)+hbar**2*k_y*gamma_0*k_y/(2*m_0)+hbar**2*k_z*gamma_0*k_z/(2*m_0)+E_0+E_v,0,-sqrt(6)*P*k_x/6-sqrt(6)*I*P*k_y/6,sqrt(6)*P*k_z/3,sqrt(2)*P*k_x/2-sqrt(2)*I*P*k_y/2,-sqrt(3)*P*k_x/3-sqrt(3)*I*P*k_y/3,sqrt(3)*P*k_z/3],[-sqrt(2)*k_x*P/2+sqrt(2)*I*k_y*P/2,0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)-hbar**2*k_x*gamma_2*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(2*m_0)-3*I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)-hbar**2*k_y*gamma_2*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(2*m_0)+3*I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)+hbar**2*k_z*gamma_2*k_z/m_0+E_v,sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)+sqrt(3)*hbar**2*k_x*k_z/(6*m_0)+sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)+sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)-sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),0,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(6)*hbar**2*k_x*k_z/(12*m_0)-sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(6)*hbar**2*k_z*k_x/(12*m_0)-sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)+sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0)],[sqrt(6)*k_z*P/3,-sqrt(6)*k_x*P/6+sqrt(6)*I*k_y*P/6,sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)-sqrt(3)*hbar**2*k_x*k_z/(6*m_0)-sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)+sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)+sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)+sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)+hbar**2*k_x*gamma_2*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(6*m_0)-I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)+hbar**2*k_y*gamma_2*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(6*m_0)+I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)-hbar**2*k_z*gamma_2*k_z/m_0+E_v,hbar**2*k_x*k_z/(3*m_0)+hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0-hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)-hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)-sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)+sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(2)*hbar**2*k_x*k_z/(12*m_0)-sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)-3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(2)*hbar**2*k_z*k_x/(12*m_0)-sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0)],[sqrt(6)*k_x*P/6+sqrt(6)*I*k_y*P/6,sqrt(6)*k_z*P/3,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-hbar**2*k_x*k_z/(3*m_0)-hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0+hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)+hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)+hbar**2*k_x*gamma_2*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(6*m_0)+I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)+hbar**2*k_y*gamma_2*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(6*m_0)-I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)-hbar**2*k_z*gamma_2*k_z/m_0+E_v,-sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)+sqrt(3)*hbar**2*k_x*k_z/(6*m_0)+sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)-sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)-sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(2)*hbar**2*k_x*k_z/(12*m_0)-sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)+3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(2)*hbar**2*k_z*k_x/(12*m_0)+sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)+sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)-sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0],[0,sqrt(2)*k_x*P/2+sqrt(2)*I*k_y*P/2,0,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)-sqrt(3)*hbar**2*k_x*k_z/(6*m_0)-sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)-sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)+sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)+sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)-hbar**2*k_x*gamma_2*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(2*m_0)+3*I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)-hbar**2*k_y*gamma_2*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(2*m_0)-3*I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)+hbar**2*k_z*gamma_2*k_z/m_0+E_v,sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(6)*hbar**2*k_x*k_z/(12*m_0)-sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(6)*hbar**2*k_z*k_x/(12*m_0)+sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0)],[-sqrt(3)*k_z*P/3,-sqrt(3)*k_x*P/3+sqrt(3)*I*k_y*P/3,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(6)*hbar**2*k_x*k_z/(12*m_0)+sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(6)*hbar**2*k_z*k_x/(12*m_0)-sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)-sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)+sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(2)*hbar**2*k_x*k_z/(12*m_0)+sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)-3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(2)*hbar**2*k_z*k_x/(12*m_0)+sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(3*m_0)-I*hbar**2*k_x*kappa*k_y/m_0-hbar**2*k_y*gamma_1*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(3*m_0)+I*hbar**2*k_y*kappa*k_x/m_0-hbar**2*k_z*gamma_1*k_z/(2*m_0)-Delta+E_v,hbar**2*k_x*k_z/(3*m_0)+hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0-hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)-hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0],[-sqrt(3)*k_x*P/3-sqrt(3)*I*k_y*P/3,sqrt(3)*k_z*P/3,-sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)+sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(2)*hbar**2*k_x*k_z/(12*m_0)+sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)+3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(2)*hbar**2*k_z*k_x/(12*m_0)-sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)+sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)-sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(6)*hbar**2*k_x*k_z/(12*m_0)+sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(6)*hbar**2*k_z*k_x/(12*m_0)+sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-hbar**2*k_x*k_z/(3*m_0)-hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0+hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)+hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(3*m_0)+I*hbar**2*k_x*kappa*k_y/m_0-hbar**2*k_y*gamma_1*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(3*m_0)-I*hbar**2*k_y*kappa*k_x/m_0-hbar**2*k_z*gamma_1*k_z/(2*m_0)-Delta+E_v]])'
display(qsymm.sympify(serialized_kp))
Hkp = qsymm.Model(serialized_kp)

Symmetry is full cubic symmetry and time reversal.

In [ ]:
sg, cg = qsymm.symmetries(Hkp, cubic_group)
print(len(sg))
print(cg)

In [ ]:
print([qsymm.groups.name_PG_elements(g) for g in sg])

Examples of how restricting parameters changes the symmetry group

In [ ]:
kp_examples = [
    {'hbar': 1},
    {'hbar': 1, 'P': 0, 'Delta': 0},
    {'hbar': 1, 'P': 0, 'Delta': 0, 'gamma_3': 0},
    {'hbar': 1, 'gamma_2': 'gamma_1', 'gamma_3': 'gamma_1', 'k_z': 0},
    {'hbar': 1, 'P': 0, 'Delta': 0, 'gamma_3': 0},
    {'hbar': 1, 'Delta': 0, 'k_z': 0},    
]

for subs in kp_examples:
    print(subs)
    Hkp = qsymm.Model(serialized_kp, locals=subs)
    %time sg, cg = qsymm.symmetries(Hkp, cubic_group)
    print(len(sg), len(cg))

# Wire with chiral symmetry

Kwant is used to generate the tight-binding model for a finite cross section wire

In [ ]:
import kwant
import matplotlib.pyplot as plt
import wire.funcs as funcs

Define symmetry candidates, in 1D the only nontrivial space-group element is inversion, besides add both TR and PH.

The $\mathbf{k\cdot p}$ Hamiltonian is discretized with a lattice constant of $a = 10$ nm. The wire and shell diameter of $r_1 = 15$ nm, $r_2 = 30$ nm used here allow the calculation in about 30 seconds, the example shown in the paper uses $r_1 = 25$ nm, $r_2 = 40$ nm and takes about 10 minutes.

The electric field is in the $z$ direction all along.

In [ ]:
candidates = {qsymm.PointGroupElement(np.eye(1), True, False, None),
              qsymm.PointGroupElement(np.eye(1), False, True, None),
              qsymm.PointGroupElement(-np.eye(1), False, False, None)
             }
candidates = qsymm.groups.generate_group(candidates)
len(candidates)

r1=15
r2=30
Vz = lambda *pos: pos[2]
color = lambda site: ('w' if (np.dot(site.pos, site.pos)) < r1**2 else 'r')

Make system with mirror symmetry and magnetic field in $x$ direction.

In [ ]:
syst_pars = dict(a=10, angle=0, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

In [ ]:
leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The system has the full 8-element symmetry group.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add magnetic field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group still contains an effective time-reversal symmetry $M_y T$

In [ ]:
%time symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add magnetic field in $y$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0.1, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group only contains $C_{2z} T$, which doesn't prevent band-tilting.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Move shell such that it breaks $M_y$

In [ ]:
syst_pars = dict(a=10, angle=45, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

Field only along the wire.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

Mirror symmetry $M_x$ is preserved.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

All symmetry is broken except for particle-hole.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]